In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_236851/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_49_5,0.998711,0.635175,0.616914,0.998196,0.000483,0.216576,0.268360,0.000780,0.009470,0.021970,0.996965,0.022165,145.272295,323.125194,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_49_9,0.998710,0.634941,0.616449,0.998064,0.000483,0.216715,0.268686,0.000837,0.009504,0.021977,0.996963,0.022172,145.271030,323.123929,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_49_8,0.998710,0.634959,0.616508,0.998079,0.000483,0.216704,0.268644,0.000830,0.009505,0.021981,0.996962,0.022176,145.270394,323.123293,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_49_4,0.998709,0.635231,0.616957,0.998207,0.000483,0.216543,0.268330,0.000775,0.009486,0.021988,0.996960,0.022183,145.269032,323.121931,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_7,0.998707,0.634923,0.616590,0.998134,0.000484,0.216726,0.268587,0.000807,0.009519,0.022003,0.996956,0.022198,145.266378,323.119278,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.455552,0.231733,0.186609,0.228400,0.203843,0.456076,0.569798,0.105203,0.247645,0.451489,-0.281721,0.455503,133.180814,311.033714,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.445021,0.224635,0.180590,0.212466,0.207786,0.460290,0.574014,0.107375,0.250087,0.455835,-0.306514,0.459887,133.142498,310.995397,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.367275,0.170429,0.124104,0.058905,0.236894,0.492469,0.613583,0.128313,0.265806,0.486717,-0.489540,0.491044,132.880287,310.733186,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.221937,0.109830,0.049137,0.191566,0.291308,0.528443,0.666099,0.110225,0.299263,0.539730,-0.831689,0.544528,132.466745,310.319645,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
